Biblioteki

In [69]:
import random
import numpy as np

Funkcje do optymalizacji

In [70]:
def sphere_function(x):
    return sum([xi**2 for xi in x])

In [71]:
def ackley_function(x):
    d = len(x)
    sum1 = sum(x**2 for x in x)
    sum2 = sum(np.cos(2 * np.pi * x) for x in x)
    return -20.0 * np.exp(-0.2 * np.sqrt(0.5 * sum1)) - np.exp(0.5 * sum2) + np.e + 20

In [72]:
def schwefel_function(x):
    return -np.sum(x * np.sin(np.sqrt(np.abs(x))))

In [73]:
def levy_function(x):
    d = len(x)
    w = [1 + (x[i] - 1) / 4 for i in range(d)]
    term1 = (np.sin(np.pi * w[0])) ** 2
    term2 = sum([(w[i] - 1) ** 2 * (1 + 10 * (np.sin(np.pi * w[i] + 1)) ** 2) for i in range(d - 1)])
    term3 = (w[d - 1] - 1) ** 2 * (1 + (np.sin(2 * np.pi * w[d - 1])) ** 2)
    return term1 + term2 + term3

In [74]:
def perm_function(x, beta=10):
    result = sum((i * (arg ** 2) ** beta) for i, arg in enumerate(x, start=1))
    return result

Funkcja inicjalizacji populacji

In [75]:
def initialize_population(pop_size, dim):
    return [[random.uniform(-5.12, 5.12) for _ in range(dim)] for _ in range(pop_size)]

Funkcja Oceny osobników

In [76]:
def fitness_function(individual, func_name):
    if func_name == 'sphere_function':
        return sphere_function(individual)
    elif func_name == 'ackley_function':
        return ackley_function(individual)
    elif func_name == 'schwefel_function':
        return schwefel_function(individual)
    elif func_name == 'levy_function':
        return levy_function(individual)
    elif func_name == 'perm_function':
        return perm_function(individual)

Funkcja selekcji turniejowa

In [77]:
def tournament_selection(population, fitness_scores, tournament_size):
    selected_individuals = []
    for _ in range(len(population)):
        tournament_indices = random.sample(range(len(population)), tournament_size)
        
        tournament_fitness_scores = [fitness_scores[i] for i in tournament_indices]

        winner_index = tournament_indices[np.argmin(tournament_fitness_scores)] # index wygranego poszczególnego turnieju

        selected_individuals.append(population[winner_index]) # gotowa lista wygranych turniejów

    return selected_individuals

Funkcja krzyżowania

In [78]:
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

Funkcja mutacji

In [79]:
def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual

Funkcja algorytmu ewolucji

In [80]:
def evolutionary_algorithm(pop_size, dim, func_name, tournament_size, num_generations, mutation_rate):
    population = initialize_population(pop_size, dim)
    history = [population.copy()]  # Historia populacji dla wizualizacji
    for _ in range(num_generations):
        fitness_scores = [fitness_function(individual, func_name) for individual in population]

        new_population = []
        selected_individuals = tournament_selection(population, fitness_scores, tournament_size)
        for _ in range(pop_size // 2):
            parent1, parent2 = random.choices(selected_individuals, k=2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.extend([child1, child2])

        population = new_population
        history.append(population.copy())  # Dodanie nowego pokolenia do historii populacji

    best_individual = min(population, key=lambda x: fitness_function(x, func_name))
    return best_individual, fitness_function(best_individual, func_name), history

Funkcja algorytmu ewolucji różnicowej*

In [81]:
def differential_evolution(pop_size, dim, func_name, num_generations, mutation_factor, crossover_rate):
    population = initialize_population(pop_size, dim)
    history = [population.copy()]  # Historia populacji dla wizualizacji
    for _ in range(num_generations):
        for i in range(pop_size):
            target_vector = population[i]
            indices = [idx for idx in range(pop_size) if idx != i]
            a, b, c = random.sample(indices, 3)
            donor_vector = np.add(population[a], mutation_factor * (np.subtract(population[b], population[c])))
            crossed_vector = []
            for j in range(dim):
                if random.random() < crossover_rate or j == random.randint(0, dim - 1):
                    crossed_vector.append(donor_vector[j])
                else:
                    crossed_vector.append(target_vector[j])
            if levy_function(crossed_vector) < levy_function(target_vector):
                population[i] = crossed_vector
        history.append(population.copy())  # Dodanie nowego pokolenia do historii populacji
    best_individual = min(population, key=lambda x: levy_function(x))
    return best_individual, levy_function(best_individual), history

Main

In [82]:
random.seed(1234)
pop_size = 100  # rozmiar populacji
dim = 3
# random.randint(3, 49) # wymiary
func_dict = {1: 'sphere_function', 2: 'ackley_function', 3: 'schwefel_function', 4: 'levy_function', 5: 'perm_function'}
func_name = "perm_function" # wybor funkcji do optymalizacji
tournament_size = 5
num_generations = 10
mutation_rate = 0.01
mutation_factor = 0.5
crossover_rate = 0.7

Wywołanie funkcji

In [83]:
best_solution, best_fitness, history = evolutionary_algorithm(pop_size, dim, func_name, tournament_size, num_generations, mutation_rate)
print("Najlepsze rozwiązanie:", best_solution)
print("Wartość funkcji celu dla najlepszego rozwiązania:", best_fitness)

Najlepsze rozwiązanie: [0.00530222 0.49792591 0.06585989]
Wartość funkcji celu dla najlepszego rozwiązania: 1.7551917296592564e-06


In [84]:
best_solution, best_fitness, history = differential_evolution(pop_size, dim, func_name, num_generations, mutation_factor, crossover_rate)
print("Najlepsze rozwiązanie:", best_solution)
print("Wartość funkcji celu dla najlepszego rozwiązania:", best_fitness)

Najlepsze rozwiązanie: [1.025314109707028, 1.0517207411956508, 0.6159329240059579]
Wartość funkcji celu dla najlepszego rozwiązania: 0.014324408857485902


In [85]:
# def sum_squares_function(x):
#     return sum(i * xi**2 for i, xi in enumerate(x, start=1))
# population = initialize_population(pop_size, dim)
# for item in population:
#     print(levy_function(item))